In [37]:
# The code was removed by DSX for sharing.

In [84]:
import pandas as pd

def covertTypes(df):
    
    df = (df.withColumn('Age',df['Age'].cast('double'))
            .withColumn('SibSp',df['SibSp'].cast('double'))
            .withColumn('Parch',df['Parch'].cast('double'))
            .withColumn('Fare',df['Fare'].cast('double'))
            .withColumn('Survived',df['Survived'].cast('double')))
    return df 
df = covertTypes(df)
test_df = covertTypes(test_df)

df.printSchema()
test_df.printSchema()

df.show()
test_df.show()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)

root
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Title: string (nullable = true)

+-----------+--------+------+------+----+-----+-----+----------------+-------+--------+-------------+
|PassengerId|Survived|Pclass|   Sex| Age|SibSp|Parch| 

In [43]:
df = df.drop('Cabin')
test_df = test_df.drop('Cabin')

In [58]:
df.printSchema()
test_df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)

root
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Title: string (nullable = true)



In [59]:
df = df.na.drop()
test_df = test_df.na.drop()

In [85]:
df.printSchema()
test_df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)

root
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Title: string (nullable = true)



In [61]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
 
## created user defined function to extract title
#def dropName
getTitleTest = udf(lambda name: name.split(',')[0].strip(),StringType())
test_df = test_df.withColumn('Title', getTitleTest(test_df['Name']))
test_df = test_df.drop('Name')


getTitle = udf(lambda name: name.split(',')[0].strip(),StringType())
df = df.withColumn('Title', getTitle(df['Name']))
df = df.drop('Name')
#return df;

#df = dropNames(df)
#test_df = dropNames(test_df)

AnalysisException: u'Cannot resolve column name "Name" among (PassengerId, Pclass, Sex, Age, SibSp, Parch, Ticket, Fare, Embarked, Survived, Title);'

In [65]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["PassengerId","Title","Pclass", "Sex", "Ticket", "Embarked"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer,encoder]


In [66]:

test_df.show()

+-----------+------+------+----+-----+-----+----------------+-------+--------+--------+-------------+
|PassengerId|Pclass|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|Survived|        Title|
+-----------+------+------+----+-----+-----+----------------+-------+--------+--------+-------------+
|        892|     3|  male|34.5|  0.0|  0.0|          330911| 7.8292|       Q|     0.0|        Kelly|
|        893|     3|female|47.0|  1.0|  0.0|          363272|    7.0|       S|     1.0|       Wilkes|
|        894|     2|  male|62.0|  0.0|  0.0|          240276| 9.6875|       Q|     0.0|        Myles|
|        895|     3|  male|27.0|  0.0|  0.0|          315154| 8.6625|       S|     0.0|         Wirz|
|        896|     3|female|22.0|  1.0|  1.0|         3101298|12.2875|       S|     1.0|     Hirvonen|
|        897|     3|  male|14.0|  0.0|  0.0|            7538|  9.225|       S|     0.0|     Svensson|
|        898|     3|female|30.0|  0.0|  0.0|          330972| 7.6292|       Q|    

In [67]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "Survived", outputCol = "label")
stages += [label_stringIdx]

In [68]:
# Transform all features into a vector using VectorAssembler
numericCols = ["Age", "SibSp","Parch", "Fare"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [69]:
df.printSchema()
test_df.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Title: string (nullable = true)

root
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Title: string (nullable = true)



In [71]:
cols = test_df.columns
# Create a Pipeline.
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.

pipeline = Pipeline(stages=stages)
def fitDf(df):
    
    pipelineModel = pipeline.fit(df)
    dataset1 = pipelineModel.transform(df)
    return dataset1

dataset = fitDf(df)
testdataSet = fitDf(test_df)

testdataSet.show()

+-----------+------+------+----+-----+-----+----------------+-------+--------+--------+-------------+----------------+-------------------+----------+-----------------+-----------+--------------+--------+-------------+-----------+-----------------+-------------+----------------+-----+--------------------+
|PassengerId|Pclass|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Embarked|Survived|        Title|PassengerIdIndex|PassengerIdclassVec|TitleIndex|    TitleclassVec|PclassIndex|PclassclassVec|SexIndex|  SexclassVec|TicketIndex|   TicketclassVec|EmbarkedIndex|EmbarkedclassVec|label|            features|
+-----------+------+------+----+-----+-----+----------------+-------+--------+--------+-------------+----------------+-------------------+----------+-----------------+-----------+--------------+--------+-------------+-----------+-----------------+-------------+----------------+-----+--------------------+
|        892|     3|  male|34.5|  0.0|  0.0|          330911| 7.8292|       Q|    

In [72]:
# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
#display(dataset)
#type(dataset)
dataset.toPandas()

testdataSet = testdataSet.select(selectedcols)
testdataSet.toPandas()

,label,features,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived,Title
0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",892,3,male,34.5,0,0,330911,7.8292,Q,0,Kelly
1,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",893,3,female,47.0,1,0,363272,7.0000,S,1,Wilkes
2,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",894,2,male,62.0,0,0,240276,9.6875,Q,0,Myles
3,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",895,3,male,27.0,0,0,315154,8.6625,S,0,Wirz
4,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",896,3,female,22.0,1,1,3101298,12.2875,S,1,Hirvonen
5,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",897,3,male,14.0,0,0,7538,9.2250,S,0,Svensson
6,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",898,3,female,30.0,0,0,330972,7.6292,Q,1,Connolly
7,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",899,2,male,26.0,1,1,248738,29.0000,S,0,Caldwell
8,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",900,3,female,18.0,0,0,2657,7.2292,C,1,Abrahim
9,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",901,3,male,21.0,2,0,A/4 48871,24.1500,S,0,Davies


In [25]:
### Randomly split data into training and test sets. set seed for reproducibility
# (trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
# print trainingData.count()
# print testData.count()

In [82]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(dataset)

predictionssss = lrModel.transform(dataset)
predictionssss.printSchema()

# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testdataSet)
predictions.printSchema()

predictionssss.select("probability").count()
#predictions.select("probability").count()
#predictions.select("prediction").count()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- Survived: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- PassengerId: string (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullabl

712

In [83]:
# View model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. For example's sake we will choose age & occupation


selected = predictions.select("label", "Age", "probability", "prediction")
selected.toPandas()
selected.show()
#predictions = predictions.select("label", "Age", "probability", "prediction")
#predictions.show()

Py4JJavaError: An error occurred while calling o4225.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 371.0 failed 10 times, most recent failure: Lost task 0.9 in stage 371.0 (TID 425, yp-spark-dal09-env5-0026, executor d6d48375-b2fd-4376-ac1a-f477005cfc4f): org.apache.spark.SparkException: Failed to execute user defined function(: (vector) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:377)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:231)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:225)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$26.apply(RDD.scala:845)
	at org.ap

In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.8459883059931782

In [26]:

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [27]:
from pyspark.ml.classification import DecisionTreeClassifier

# Create initial Decision Tree Model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=3)

# Train model with Training Data
dtModel = dt.fit(trainingData)

In [28]:
print "numNodes = ", dtModel.numNodes
print "depth = ", dtModel.depth

numNodes =  15
depth =  3


In [29]:
# Make predictions on test data using the Transformer.transform() method.
predictions = dtModel.transform(testData)

In [30]:
selected = predictions.select("label", "prediction", "probability", "Age")
selected.show()

+-----+----------+--------------------+----+
|label|prediction|         probability| Age|
+-----+----------+--------------------+----+
|  0.0|       1.0|         [0.45,0.55]| 2.0|
|  0.0|       1.0|         [0.45,0.55]| 8.0|
|  0.0|       0.0|[0.88636363636363...|20.0|
|  0.0|       0.0|[0.88636363636363...|28.0|
|  0.0|       0.0|[0.88636363636363...|19.0|
|  0.0|       0.0|[0.61111111111111...|45.0|
|  0.0|       0.0|[0.88636363636363...|30.0|
|  0.0|       0.0|[0.88636363636363...|48.0|
|  0.0|       1.0|         [0.45,0.55]|11.0|
|  0.0|       0.0|[0.88636363636363...|22.0|
|  0.0|       1.0|         [0.45,0.55]| 2.0|
|  0.0|       1.0|         [0.45,0.55]| 7.0|
|  0.0|       0.0|       [0.875,0.125]| 3.0|
|  0.0|       1.0|[0.40322580645161...|21.0|
|  0.0|       0.0|[0.88636363636363...|36.0|
|  0.0|       0.0|[0.88636363636363...|29.0|
|  0.0|       0.0|[0.88636363636363...|17.0|
|  0.0|       0.0|[0.88636363636363...|37.0|
|  0.0|       0.0|       [0.875,0.125]| 2.0|
|  0.0|   

In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.8239808348221538

In [32]:
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [33]:
predictions = rfModel.transform(testData)

In [34]:
# View model's predictions and probabilities of each prediction class
selected = predictions.select("label", "prediction", "probability", "Age")
selected.show()

+-----+----------+--------------------+----+
|label|prediction|         probability| Age|
+-----+----------+--------------------+----+
|  0.0|       0.0|[0.62972458353542...| 2.0|
|  0.0|       0.0|[0.65428549005100...| 8.0|
|  0.0|       0.0|[0.70355576696306...|20.0|
|  0.0|       0.0|[0.68736980700028...|28.0|
|  0.0|       0.0|[0.70355576696306...|19.0|
|  0.0|       0.0|[0.57816693814403...|45.0|
|  0.0|       0.0|[0.65910558916788...|30.0|
|  0.0|       0.0|[0.70355576696306...|48.0|
|  0.0|       0.0|[0.67105109354999...|11.0|
|  0.0|       0.0|[0.70355576696306...|22.0|
|  0.0|       0.0|[0.66164945976849...| 2.0|
|  0.0|       0.0|[0.68621036628407...| 7.0|
|  0.0|       0.0|[0.61090255946120...| 3.0|
|  0.0|       0.0|[0.63226818574442...|21.0|
|  0.0|       0.0|[0.70355576696306...|36.0|
|  0.0|       0.0|[0.68736980700028...|29.0|
|  0.0|       0.0|[0.70355576696306...|17.0|
|  0.0|       0.0|[0.62660091575481...|37.0|
|  0.0|       0.0|[0.62892879089358...| 2.0|
|  0.0|   

In [35]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

0.8243462725353256

In [36]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

layers = [4, 5, 4, 1]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(dataset)

Py4JJavaError: An error occurred while calling o933.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 107.0 failed 10 times, most recent failure: Lost task 0.9 in stage 107.0 (TID 113, yp-spark-dal09-env5-0026, executor c1ebe4b5-e896-4500-97eb-89bbcaaf42ca): java.lang.ArrayIndexOutOfBoundsException: Array index out of range: 1
	at org.apache.spark.ml.classification.LabelConverter$.encodeLabeledPoint(MultilayerPerceptronClassifier.scala:121)
	at org.apache.spark.ml.classification.MultilayerPerceptronClassifier$$anonfun$3.apply(MultilayerPerceptronClassifier.scala:238)
	at org.apache.spark.ml.classification.MultilayerPerceptronClassifier$$anonfun$3.apply(MultilayerPerceptronClassifier.scala:238)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at scala.collection.Iterator$GroupedIterator.takeDestructively(Iterator.scala:1076)
	at scala.collection.Iterator$GroupedIterator.go(Iterator.scala:1091)
	at scala.collection.Iterator$GroupedI